In [1]:
import os
import pandas as pd
import ftplib
import SimpleITK as sitk

In [2]:
def getFile(ftp, base_path, filename):
    try:
        ftp.retrbinary("RETR " + filename ,open(os.path.join(base_path,filename), 'wb').write)
    except:
        print("Error")

In [3]:
output_directory = r'C:\Users\LydiaN\OneDrive - Personal\OneDrive\Work\DigitalAssets\BICCNMorphologyInventory\u19zeng'
inventory_path = os.path.join(output_directory,'sample_inventory_with_image_info.csv')
inventory = pd.read_csv(inventory_path)
inventory.set_index('Sample ID', inplace=True)

inventory.head()

,HUST Sample ID,Sample Type (CV),Species (CV),Species NCBI Taxonomy ID,Parent Specimen ID,Parent Specimen Type (CV),Subject ID,Age,Sex (CV),Genotype,...,Directories,TXT Files,JSON Files,x_resolution,y_resolution,z_resolution,x_size,y_size,z_size,rotation
Sample ID,,,,,,,,,,,,,,,,,,,,,
mouseID_210254-15257,15257,whole brain,mouse,NCBI:txid10090,NaN,NaN,210254,22w,female,Pvalb-T2A-CreERT2/wt;Ai139(TIT2L-GFP-ICL-TPT)/wt,...,.;..;green;red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mouseID_236174-16124,16124,whole brain,mouse,NCBI:txid10090,NaN,NaN,236174,21w,male,Gnb4-IRES2-CreERT2/wt;Ai139(TIT2L-GFP-ICL-TPT)...,...,.;..;green;red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mouseID_297974-17109,17109,whole brain,mouse,NCBI:txid10090,NaN,NaN,297974,14w,male,Gnb4-IRES2-CreERT2/wt;Ai140(TIT2L-GFP-ICL-tTA2...,...,.;..;green;red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mouseID_314107-17300,17300,whole brain,mouse,NCBI:txid10090,NaN,NaN,314107,48w,male,Plxnd1-CreER/wt;Ai82(TITL-GFP)/Ai140(TIT2L-GFP...,...,.;..;green;red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mouseID_321237-17302,17302,whole brain,mouse,NCBI:txid10090,NaN,NaN,321237,29w,male,Tnnt1-IRES2-CreERT2/wt;Ai82(TITL-GFP)/Ai140(TI...,...,.;..;green;red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
cols = ['sample_id', 'hust_imaging_id', 'labtrack_mouse_id', 'channel', 'image_type', 'resolution', 'number_of_images', 'first_image_path', 'last_image_path']
file_inventory = pd.DataFrame( columns = cols )

In [5]:
def get_dir_info(ftp) :
    
    data = []
    ftp.dir(data.append)
    data = [x.split() for x in data if x != '']
    data = pd.DataFrame(data)
    
    return data

def check_for_directory(data,dirname) :
    has_dir = False
    for dindex, drow in data.iterrows() :
        if drow[8] == dirname :
            has_dir = True
    return has_dir

In [6]:
request_list = [
18865,
18866,
18872,
182709,
182711,
182712,
182728,
191797,
191798,
191799,
191809,
191810,
191811,
191820,
192335,
192348,
196470,
196471,
196472,
196473,
196474,
201591,
201593,
201594,
201602,
201604,
201605,
201606
]

In [7]:
count = 0
for iindex, irow in inventory.iterrows() :
    
    if not int(irow['HUST Sample ID']) in request_list :
        continue
        
    ftp = ftplib.FTP(host="download.brainimagelibrary.org")
    ftp.login()

    home_dir = ftp.pwd()
    remote_base = "/bil/data/biccn/zeng/luo/fMOST"
    
    print(iindex)
    
    # specimen output directory
    spath = os.path.join(output_directory,iindex)
    
    # get listing of remote directory
    rpath = remote_base + "/" + iindex
    
    clist = ['green','red']
    tlist = ['raw','mip']
    rlist = ['fullres','lowres']
    
    clist = ['green']
    tlist = ['raw']
    rlist = ['fullres']
    
    for c in clist :
        for t in tlist :
            for r in rlist :
                
                ftp.cwd( rpath )
                dlist = [c,t,r]
                print( "- " + str(dlist))
    
                for d in dlist :
                    data = get_dir_info(ftp)
                    has_dir = check_for_directory( data, d )
                    if not has_dir :
                        break
                    ftp.cwd(d)
        
                if not has_dir :
                    print("-- Directory not found - skipping")
                    continue        
    
                rec = {}
                rec['sample_id'] = iindex
                rec['hust_imaging_id'] = irow['HUST Sample ID']
                rec['labtrack_mouse_id'] = irow['Subject ID']
                rec['channel'] = c
                rec['image_type'] = t
                rec['resolution'] = r
 
                if 1 :
                    print("-- Start: dir listing")
                    data = get_dir_info(ftp)
                    print("-- End: dir listing")

                    matched = ['tif' in x for x in data[8]]
                    filtered = data[matched]
                    ordered = filtered.sort_values(by=8)
                    if len(ordered) > 0 :
                        first_file = ordered.loc[ordered.index[0], 8]
                        last_file = ordered.loc[ordered.index[-1], 8]
                        num_files = len(ordered)
                        print(num_files, first_file, last_file)
                    
                        rec['number_of_images'] = num_files
                        rec['first_image_path'] = first_file
                        rec['last_image_path'] = last_file
                    
                file_inventory = file_inventory.append(rec, ignore_index=True)

    ftp.quit()


mouseID_439309-191798
- ['green', 'raw', 'fullres']
-- Start: dir listing
-- End: dir listing
10711 191798_00001_CH1.tif 191798_10711_CH1.tif
mouseID_439311-191799
- ['green', 'raw', 'fullres']
-- Start: dir listing
-- End: dir listing
11025 191799_00001_CH1.tif 191799_11025_CH1.tif
mouseID_394496-182728
- ['green', 'raw', 'fullres']
-- Start: dir listing
-- End: dir listing
10350 182728_00001_CH1.tif 182728_10350_CH1.tif
mouseID_426812-191820
- ['green', 'raw', 'fullres']
-- Start: dir listing
-- End: dir listing
10998 191820_00001_CH1.tif 191820_10998_CH1.tif
mouseID_444771-192348
- ['green', 'raw', 'fullres']
-- Start: dir listing
-- End: dir listing
12216 192348_00001_CH1.tif 192348_12216_CH1.tif
mouseID_391751-18872
- ['green', 'raw', 'fullres']
-- Start: dir listing
-- End: dir listing
10370 18872_00001_CH1.tif 18872_10370_CH1.tif
mouseID_372164-182712
- ['green', 'raw', 'fullres']
-- Start: dir listing
-- End: dir listing
10764 182712_00001_CH1.tif 182712_10764_CH1.tif
mouseID_3

In [8]:
file_inventory_path = os.path.join( output_directory, 'file_inventory_part_3.csv')
file_inventory.to_csv( file_inventory_path, index=False )

In [9]:
file_inventory

,sample_id,hust_imaging_id,labtrack_mouse_id,channel,image_type,resolution,number_of_images,first_image_path,last_image_path
0,mouseID_439309-191798,191798,439309,green,raw,fullres,10711,191798_00001_CH1.tif,191798_10711_CH1.tif
1,mouseID_439311-191799,191799,439311,green,raw,fullres,11025,191799_00001_CH1.tif,191799_11025_CH1.tif
2,mouseID_394496-182728,182728,394496,green,raw,fullres,10350,182728_00001_CH1.tif,182728_10350_CH1.tif
3,mouseID_426812-191820,191820,426812,green,raw,fullres,10998,191820_00001_CH1.tif,191820_10998_CH1.tif
4,mouseID_444771-192348,192348,444771,green,raw,fullres,12216,192348_00001_CH1.tif,192348_12216_CH1.tif
5,mouseID_391751-18872,18872,391751,green,raw,fullres,10370,18872_00001_CH1.tif,18872_10370_CH1.tif
6,mouseID_372164-182712,182712,372164,green,raw,fullres,10764,182712_00001_CH1.tif,182712_10764_CH1.tif
7,mouseID_375886-182709,182709,375886,green,raw,fullres,11017,182709_00001_CH1.tif,182709_11017_CH1.tif
8,mouseID_491992-196470,196470,491992,green,raw,fullres,11347,196470_00001_CH1.tif,196470_11347_CH1.tif
9,mouseID_491993-196471,196471,491993,green,raw,fullres,11096,196471_00001_CH1.tif,196471_11096_CH1.tif
